In [1]:
using ITensors
using ITensorChemistry

# Find r

In [2]:
function create_molecule(m::String, r::Float64, rep::Int64)
    s = []
    for j in 1:rep
        push!(s,Atom(m, j*r , 0.0, 0.0))
    end
    return Molecule(s)
end

create_molecule (generic function with 1 method)

In [3]:
function energy_at_bond(r,N)
  # define molecule geometry
  molecule = create_molecule("H",r,N)
  # build electronic hamiltonian and solve HF
  hf = molecular_orbital_hamiltonian(molecule; basis="sto-3g")
  hamiltonian = hf.hamiltonian
  hartree_fock_state = hf.hartree_fock_state
  hartree_fock_energy = hf.hartree_fock_energy
  # hilbert space
  s = siteinds("Electron", N; conserve_qns=true)
  H = MPO(hamiltonian, s)
  # initialize MPS to HF state
  ψhf = MPS(s, hartree_fock_state)
  # run dmrg
  dmrg_kwargs = (;
    nsweeps=2,
    maxdim=[10,20,30,40,50,100],
    cutoff=1e-8,
    noise=[1e-6, 1e-7, 1e-8, 0.0],
  )
  dmrg_energy, _ = dmrg(H, ψhf; nsweeps=2, outputlevel=0)
  return hartree_fock_energy, dmrg_energy
end

energy_at_bond (generic function with 1 method)

In [4]:
energy_at_bond(0.1,2)[1]

converged SCF energy = 2.71588739329275
RHF Energy (Ha): 2.7158873932927525


2.7158873932927525

In [13]:
# bond distances
r⃗ = 0.8:0.01:1
# energies = []
hartee_energy = []
dmrg_energy = []
for r in r⃗
    push!(hartee_energy, energy_at_bond(r,4)[1])
    push!(dmrg_energy, energy_at_bond(r,4)[2])
#   push!(energies, (r,energy_at_bond(r,4)))
end

converged SCF energy = -2.12138675587022
RHF Energy (Ha): -2.121386755870221
converged SCF energy = -2.12138675587022
RHF Energy (Ha): -2.1213867558702226
converged SCF energy = -2.12345177439825
RHF Energy (Ha): -2.123451774398254
converged SCF energy = -2.12345177439825
RHF Energy (Ha): -2.1234517743982533
converged SCF energy = -2.1250629749356
RHF Energy (Ha): -2.1250629749355956
converged SCF energy = -2.1250629749356
RHF Energy (Ha): -2.1250629749355965
converged SCF energy = -2.12624469644404
RHF Energy (Ha): -2.1262446964440445
converged SCF energy = -2.12624469644405
RHF Energy (Ha): -2.126244696444048
converged SCF energy = -2.12701986115409
RHF Energy (Ha): -2.127019861154087
converged SCF energy = -2.12701986115409
RHF Energy (Ha): -2.1270198611540887
converged SCF energy = -2.12741006395107
RHF Energy (Ha): -2.1274100639510727
converged SCF energy = -2.12741006395107
RHF Energy (Ha): -2.127410063951071
converged SCF energy = -2.12743565642466
RHF Energy (Ha): -2.1274356564

In [14]:
length(dmrg_energy), length(hartee_energy)

(21, 21)

In [15]:
r⃗[findmin(dmrg_energy)[2]], r⃗[findmin(hartee_energy)[2]]

(0.87, 0.86)

In [16]:
findmin(dmrg_energy) , findmin(hartee_energy)

((-2.15714737214169, 8), (-2.127435656424658, 7))

In [12]:
# function min_energy(energy)
#     dm_energy = []
#     for j in energy
#         push!(dm_energy, j[2][2])
#     end
#     min = minimum(dm_energy)
#     for j in energy
#         if j[2][2] == min
#             return j
#         end
#     end
# end

# compute MPS

After finding optimal $r$ we can use it in it the main algorithm

In [17]:
function energy_at_bond(r)
  # define molecule geometry
  molecule = create_molecule("H",r,4)
  # build electronic hamiltonian 
  hf = molecular_orbital_hamiltonian(molecule; basis="sto-3g")
  hamiltonian = hf.hamiltonian
  hartree_fock_state = hf.hartree_fock_state
  hartree_fock_energy = hf.hartree_fock_energy
  # hilbert space
  s = siteinds("Electron", 4; conserve_qns=true)
  H = MPO(hamiltonian, s)
  # initialize MPS to HF state
  ψhf = MPS(s, hartree_fock_state)
  # run dmrg
  dmrg_kwargs = (;
    nsweeps=10,
    maxdim=[10,20,30,40,50,100],
    cutoff=1e-8,
    noise=[1e-6, 1e-7, 1e-8, 0.0],
  )
  dmrg_energy, _ = dmrg(H, ψhf; nsweeps=10, outputlevel=0)
  return hartree_fock_energy, dmrg_energy
end

energy_at_bond (generic function with 2 methods)

In [18]:
# some changes to this func on the line 22
function energy_at_bond(N,r)
  # define molecule geometry
  molecule = create_molecule("H",r,N)
  # build electronic hamiltonian 
  hf = molecular_orbital_hamiltonian(molecule; basis="sto-3g")
  hamiltonian = hf.hamiltonian
  hartree_fock_state = hf.hartree_fock_state
  hartree_fock_energy = hf.hartree_fock_energy
  # hilbert space
  s = siteinds("Electron", N; conserve_qns=true)
  H = MPO(hamiltonian, s)
  # initialize MPS to HF state
  ψhf = MPS(s, hartree_fock_state)
  # run dmrg
  dmrg_kwargs = (;
    nsweeps=10,
    maxdim=[10,20,30,40,50,100],
    cutoff=1e-8,
    noise=[1e-6, 1e-7, 1e-8, 0.0],
  )
  dm_e, dm_st = dmrg(H, ψhf; nsweeps=10, outputlevel=0)
#   return hartree_fock_energy, dmrg_energy
    return dm_e, dm_st
end

energy_at_bond (generic function with 2 methods)

In [24]:
e2, ψ2 = energy_at_bond(2, 0.87);
e4, ψ4 = energy_at_bond(4, 0.87);
# e6, ψ6 = energy_at_bond(6, 0.87);
# e8, ψ8 = energy_at_bond(8, 0.87);
# e10, ψ10 = energy_at_bond(10, 0.87);
# e12, ψ12 = energy_at_bond(12, 0.87);

converged SCF energy = -1.0985065850866
RHF Energy (Ha): -1.098506585086596
converged SCF energy = -2.12711582594087
RHF Energy (Ha): -2.127115825940872


In [27]:
println(typeof(ψ2), "\t ",length(ψ2))
println(typeof(ψ4), "\t ",length(ψ4))

MPS	 2
MPS	 4


In [23]:
ψ41 = ψ4[1:2];
ψ42 = ψ4[3:4];


2

# Compute overlap

In [14]:
v = nothing

In [28]:
v = ITensor(1.0)
for j in 1:4
    if j <= 2
        v = v*dag(ψ4[j])*ψ2[j]
#     else
#         v = v*dag(dmrg_state4[j])*dmrg_state22[j-2]
    end
end
println(inner(v,v))

1.0
